In [1]:
import paddle

print('飞桨框架内置模型：', paddle.vision.models.__all__)


飞桨框架内置模型： ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'VGG', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'MobileNetV1', 'mobilenet_v1', 'MobileNetV2', 'mobilenet_v2', 'LeNet']


In [2]:
# 模型组网并初始化网络
lenet = paddle.vision.models.LeNet(num_classes=10)

# 可视化模型组网结构和参数
paddle.summary(lenet,(1, 1, 28, 28))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1       [[1, 1, 28, 28]]      [1, 6, 28, 28]          60       
    ReLU-1        [[1, 6, 28, 28]]      [1, 6, 28, 28]           0       
  MaxPool2D-1     [[1, 6, 28, 28]]      [1, 6, 14, 14]           0       
   Conv2D-2       [[1, 6, 14, 14]]     [1, 16, 10, 10]         2,416     
    ReLU-2       [[1, 16, 10, 10]]     [1, 16, 10, 10]           0       
  MaxPool2D-2    [[1, 16, 10, 10]]      [1, 16, 5, 5]            0       
   Linear-1          [[1, 400]]            [1, 120]           48,120     
   Linear-2          [[1, 120]]            [1, 84]            10,164     
   Linear-3          [[1, 84]]             [1, 10]              850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward

{'total_params': 61610, 'trainable_params': 61610}

经典模型可以满足一些简单深度学习任务的需求，然后更多情况下，需要使用深度学习框架构建一个自己的神经网络，这时可以使用飞桨框架 paddle.nn 下的 API 构建网络，该目录下定义了丰富的神经网络层和相关函数 API，如卷积网络相关的 Conv1D、Conv2D、Conv3D，循环神经网络相关的 RNN、LSTM、GRU 等，方便组网调用，详细清单可在 API 文档 中查看。

飞桨提供继承类（class）的方式构建网络，并提供了几个基类，如：paddle.nn.Sequential、 paddle.nn.Layer 等，构建一个继承基类的子类，并在子类中添加层（layer，如卷积层、全连接层等）可实现网络的构建，不同基类对应不同的组网方式，本节介绍如下两种常用方法：

使用 paddle.nn.Sequential 组网：构建顺序的线性网络结构（如 LeNet、AlexNet 和 VGG）时，可以选择该方式。相比于 Layer 方式 ，Sequential 方式可以用更少的代码完成线性网络的构建。

使用 paddle.nn.Layer 组网（推荐）：构建一些比较复杂的网络结构时，可以选择该方式。相比于 Sequential 方式，Layer 方式可以更灵活地组建各种网络结构。Sequential 方式搭建的网络也可以作为子网加入 Layer 方式的组网中。

## 使用 paddle.nn.Sequential 组网[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#paddle-nn-sequential "永久链接至标题")

构建顺序的线性网络结构时，可以选择该方式，只需要按模型的结构顺序，一层一层加到 [paddle.nn.Sequential](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Sequential_cn.html#sequential) 子类中即可。

参照前面图 1 所示的 LeNet 模型结构，构建该网络结构的代码如下

使用 Sequential 组网时，会自动按照层次堆叠顺序完成网络的前向计算过程，简略了定义前向计算函数的代码。由于 Sequential 组网只能完成简单的线性结构模型，所以对于需要进行分支判断的模型需要使用 paddle.nn.Layer 组网方式实现。

In [4]:
from paddle import nn

# 使用 paddle.nn.Sequential 构建 LeNet 模型
lenet_Sequential = nn.Sequential(
    nn.Conv2D(1, 6, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2D(2, 2),
    nn.Conv2D(6, 16, 5, stride=1, padding=0),
    nn.ReLU(),
    nn.MaxPool2D(2, 2),
    nn.Flatten(),
    nn.Linear(400, 120),
    nn.Linear(120, 84),
    nn.Linear(84, 10)
)
# 可视化模型组网结构和参数
paddle.summary(lenet_Sequential,(1, 1, 28, 28))


---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-5       [[1, 1, 28, 28]]      [1, 6, 28, 28]          60       
    ReLU-5        [[1, 6, 28, 28]]      [1, 6, 28, 28]           0       
  MaxPool2D-5     [[1, 6, 28, 28]]      [1, 6, 14, 14]           0       
   Conv2D-6       [[1, 6, 14, 14]]     [1, 16, 10, 10]         2,416     
    ReLU-6       [[1, 16, 10, 10]]     [1, 16, 10, 10]           0       
  MaxPool2D-6    [[1, 16, 10, 10]]      [1, 16, 5, 5]            0       
   Flatten-4      [[1, 16, 5, 5]]          [1, 400]              0       
   Linear-7          [[1, 400]]            [1, 120]           48,120     
   Linear-8          [[1, 120]]            [1, 84]            10,164     
   Linear-9          [[1, 84]]             [1, 10]              850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
----------------------------------------

{'total_params': 61610, 'trainable_params': 61610}

## 使用 paddle.nn.Layer 组网[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#paddle-nn-layer "永久链接至标题")

构建一些比较复杂的网络结构时，可以选择该方式，组网包括三个步骤：

1. 创建一个继承自 [paddle.nn.Layer](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#layer) 的类；

2. 在类的构造函数 `__init__` 中定义组网用到的神经网络层（layer）；

3. 在类的前向计算函数 `forward` 中使用定义好的 layer 执行前向计算。


仍然以 LeNet 模型为例，使用 paddle.nn.Layer 组网的代码如下：

In [11]:
# 使用 Subclass 方式构建 LeNet 模型
class LeNet(nn.Layer):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        # 构建 features 子网，用于对输入图像进行特征提取
        self.features = nn.Sequential(
            nn.Conv2D(1, 6, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2D(2, 2),
            nn.Conv2D(6, 16, 5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2D(2, 2))
        # 构建 linear 子网，用于分类
        if num_classes > 0:
            self.linear = nn.Sequential(
                nn.Linear(400, 120),
                nn.Linear(120, 84),
                nn.Linear(84, num_classes)
            )
    # 执行前向计算
    def forward(self, inputs):
        x = self.features(inputs)

        if self.num_classes > 0:
            x = paddle.flatten(x, 1)
            x = self.linear(x)
        return x
lenet_SubClass = LeNet()

# 可视化模型组网结构和参数
params_info = paddle.summary(lenet_SubClass,(1, 1, 28, 28))
print(params_info)


---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-10      [[1, 1, 28, 28]]      [1, 6, 28, 28]          60       
    ReLU-10       [[1, 6, 28, 28]]      [1, 6, 28, 28]           0       
 MaxPool2D-10     [[1, 6, 28, 28]]      [1, 6, 14, 14]           0       
   Conv2D-11      [[1, 6, 14, 14]]     [1, 16, 10, 10]         2,416     
    ReLU-11      [[1, 16, 10, 10]]     [1, 16, 10, 10]           0       
 MaxPool2D-11    [[1, 16, 10, 10]]      [1, 16, 5, 5]            0       
   Linear-14         [[1, 400]]            [1, 120]           48,120     
   Linear-15         [[1, 120]]            [1, 84]            10,164     
   Linear-16         [[1, 84]]             [1, 10]              850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward

### Conv2D[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#conv2d "永久链接至标题")

[Conv2D](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Conv2D_cn.html#conv2d) （二维卷积层）主要用于对输入的特征图进行卷积操作，广泛用于深度学习网络中。Conv2D 根据输入、卷积核、步长（stride）、填充（padding）、空洞大小（dilations）等参数计算输出特征层大小。输入和输出是 NCHW 或 NHWC 格式，其中 N 是 batchsize 大小，C 是通道数，H 是特征高度，W 是特征宽度。

In [12]:
x = paddle.uniform((2, 3, 8, 8), dtype='float32', min=-1., max=1.)

conv = nn.Conv2D(3, 6, (3, 3), stride=2) # 卷积层输入通道数为3，输出通道数为6，卷积核尺寸为3*3，步长为2
y = conv(x) # 输入数据x
y = y.numpy()
print(y.shape)


(2, 6, 3, 3)


### MaxPool2D[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#maxpool2d "永久链接至标题")

[MaxPool2D](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/MaxPool2D_cn.html#maxpool2d) （二维最大池化层）主要用于缩小特征图大小，根据 `kernel_size` 参数指定的窗口大小，对窗口内特征图进行取最大值的操作。

In [13]:
x = paddle.uniform((2, 3, 8, 8), dtype='float32', min=-1., max=1.)

pool = nn.MaxPool2D(3, stride=2) # 池化核尺寸为3*3，步长为2
y = pool(x) #输入数据x
y = y.numpy()
print(y.shape)


(2, 3, 3, 3)


### Linear[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#linear "永久链接至标题")

[Linear](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Linear_cn.html#linear) （全连接层）中每个神经元与上一层的所有神经元相连，实现对前一层的线性组合和线性变换。在卷积神经网络分类任务中，输出分类结果之前，通常采用全连接层对特征进行处理。

In [14]:
x = paddle.uniform((2, 6), dtype='float32', min=-1., max=1.)
linear = paddle.nn.Linear(6, 4)
y = linear(x)
print(y.shape)


[2, 4]


### ReLU[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#relu "永久链接至标题")

[ReLU](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/ReLU_cn.html#relu) 是深度学习任务中常用的激活层，主要用于对输入进行非线性变换。ReLU 将输入中小于 0 的部分变为 0，大于 0 的部分保持不变。

In [15]:
x = paddle.to_tensor([-2., 0., 1.])
relu = paddle.nn.ReLU()
y = relu(x)
print(y)


Tensor(shape=[3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [0., 0., 1.])


## 模型的参数（Parameter）[¶](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_cn.html#parameter "永久链接至标题")

在飞桨框架中，可通过网络的 [parameters()](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#parameters) 和 [named\_parameters()](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#named_parameters) 方法获取网络在训练期间优化的所有参数（权重 weight 和偏置 bias），通过这些方法可以实现对网络更加精细化的控制，如设置某些层的参数不更新。

下面这段示例代码，通过 `named_parameters()` 获取了 LeNet 网络所有参数的名字和值，打印出了参数的名字（name）和形状（shape）。

In [10]:
 for name, param in lenet.named_parameters():
    print(f"Layer: {name} | Size: {param.shape}")


Layer: features.0.weight | Size: [6, 1, 3, 3]
Layer: features.0.bias | Size: [6]
Layer: features.3.weight | Size: [16, 6, 5, 5]
Layer: features.3.bias | Size: [16]
Layer: fc.0.weight | Size: [400, 120]
Layer: fc.0.bias | Size: [120]
Layer: fc.1.weight | Size: [120, 84]
Layer: fc.1.bias | Size: [84]
Layer: fc.2.weight | Size: [84, 10]
Layer: fc.2.bias | Size: [10]


In [18]:
 for name in lenet.parameters():
    print(f"Layer: {name} |")

Layer: Parameter containing:
Tensor(shape=[6, 1, 3, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[[[-0.19695179,  0.57811522,  0.29227942],
          [ 0.08942306,  0.28864464,  0.34071708],
          [ 0.15786977, -0.63316137,  0.30367732]]],


        [[[ 0.52591950, -0.17076568, -0.46024197],
          [-0.24584584,  0.27417296, -0.66235065],
          [ 0.36220843,  0.01550483,  0.35309982]]],


        [[[-0.02318917,  0.09696703,  0.73748082],
          [ 0.52494359,  0.71905369,  0.12482166],
          [ 0.15170661,  0.03854354, -0.26169708]]],


        [[[ 0.05439302,  0.10537121, -0.07011510],
          [ 0.32484215, -0.45675200, -0.50786769],
          [ 0.58705974,  1.07583737, -0.01500405]]],


        [[[ 0.21232437,  0.12764770, -1.21040213],
          [-0.54420960, -0.25622305,  0.48257440],
          [-0.85885727,  0.02369257,  0.81230468]]],


        [[[ 0.51790011,  0.52297723,  0.15236698],
          [-0.50385445, -0.67869157, -0.95017326],
 

In [21]:
 for name in lenet.state_dict():
    print(f"Layer: {name} |Size:{lenet.state_dict()[name].shape}")

Layer: features.0.weight |Size:[6, 1, 3, 3]
Layer: features.0.bias |Size:[6]
Layer: features.3.weight |Size:[16, 6, 5, 5]
Layer: features.3.bias |Size:[16]
Layer: fc.0.weight |Size:[400, 120]
Layer: fc.0.bias |Size:[120]
Layer: fc.1.weight |Size:[120, 84]
Layer: fc.1.bias |Size:[84]
Layer: fc.2.weight |Size:[84, 10]
Layer: fc.2.bias |Size:[10]
